1º Projeto — Área de Logística (Vendas por Localidade)


In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DecimalType

catalogo = "rocketlab"
silver_schema = "silver"
gold_schema = "gold"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalogo}.{gold_schema}")

spark = SparkSession.builder.appName("Atividade2").getOrCreate()


In [0]:
s_ped_total = spark.read.table(f'{catalogo}.{silver_schema}.ft_pedido_total') 
s_cs = spark.read.table(f'{catalogo}.{silver_schema}.ft_consumidores')

f_vendas_cs_local = (
    s_ped_total
    .join(
        s_cs, 
        'id_consumidor'
        )
    .select('id_pedido',
            'id_consumidor',
             F.col('valor_total_pago_brl').cast(DecimalType(12, 2)), 
             'cidade', 
             'estado', 
             'data_pedido'
        )
    .withColumn('data_ingestao', F.current_timestamp())
    )
(
    f_vendas_cs_local.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalogo}.{gold_schema}.ft_vendas_consumidor_local")
)
print('table "ft_vendas_consumidor_local" created')


In [0]:
%sql
CREATE OR REPLACE VIEW rocketlab.gold.view_total_compras_por_consumidor AS
SELECT 
    cidade, 
    estado, 
    COUNT(DISTINCT id_consumidor) as total_consumidores_localidade, 
    SUM(valor_total_pago_brl) as valor_total_localidade,
    CURRENT_TIMESTAMP AS data_ingestao
FROM rocketlab.gold.ft_vendas_consumidor_local
GROUP BY cidade, estado

In [0]:
%sql
SELECT estado, SUM(total_consumidores_localidade) as total_consumidores_localidade, SUM(valor_total_localidade) as valor_total_localidade
FROM rocketlab.gold.view_total_compras_por_consumidor
GROUP BY estado

2º Projeto — Área de Logística (Análise de Atrasos de Entregas)

In [0]:
s_pd = spark.read.table(f'{catalogo}.{silver_schema}.ft_pedidos').alias('s_pd')
s_cs = spark.read.table(f'{catalogo}.{silver_schema}.ft_consumidores').alias('s_cs')
s_i_pd = spark.read.table(f'{catalogo}.{silver_schema}.ft_itens_pedidos').alias('s_i_pd')

# TABELA FATO ATRASOS PEDISO LOCAL VENDEDOR 
atrasos = (
    s_pd
    .join(s_cs, 'id_consumidor')
    .join(s_i_pd, 'id_pedido')
    .select(
        s_pd.id_pedido,
        s_i_pd.id_vendedor,
        s_pd.id_consumidor,
        s_pd.entrega_no_prazo,
        s_pd.tempo_entrega_dias,
        s_pd.tempo_entrega_estimado_dias,
        s_cs.cidade,
        s_cs.estado,
    )
    .withColumn('data_ingestao', F.current_timestamp())
)
(
    atrasos.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalogo}.{gold_schema}.ft_atrasos_pedidos_local_vendedor")
)
print('table "ft_atrasos_pedidos_local_vendedor" created')



In [0]:
%sql
CREATE OR REPLACE VIEW rocketlab.gold.view_tempo_medio_entrega_localidade AS
SELECT 
  cidade, 
  estado,
  AVG(tempo_entrega_dias) AS tempo_medio_entrega_dias,
  AVG(tempo_entrega_estimado_dias) AS tempo_medio_entrega_estimado_dias,
  CASE 
    WHEN tempo_medio_entrega_dias > tempo_medio_entrega_estimado_dias THEN "sim"
    ELSE "nao"
  END AS entrega_maior_que_estimado,
  CURRENT_TIMESTAMP AS data_ingestao
FROM rocketlab.gold.ft_atrasos_pedidos_local_vendedor 
GROUP BY cidade, estado

In [0]:
%sql
CREATE OR REPLACE VIEW rocketlab.gold.view_vendedor_pontualidade AS
SELECT 
  COUNT(DISTINCT id_pedido) as total_pedidos,
  SUM(CASE 
        WHEN lower(entrega_no_prazo) = 'nao' THEN 1
        ELSE 0
    END) AS total_atrasados,
    ROUND(total_atrasados / total_pedidos, 2) AS percentual_atraso,
    CURRENT_TIMESTAMP AS data_ingestao
FROM rocketlab.gold.ft_atrasos_pedidos_local_vendedor 
GROUP BY id_vendedor

3º Projeto — Área Comercial (Análises de Vendas por Período)

In [0]:
# DM TEMPO

#strings (ano-mes-dia h-m-s)
data_inicio = '2017-01-01 00:00:00'
data_fim = '2020-12-31 00:00:00'

# Como a função do spark retornar nomes em ingles, melhor usar um map
day_name_dict = {
    1: 'domingo',
    2: 'segunda',
    3: 'terca', # proposital o 'c'
    4: 'quarta',
    5: 'quinta',
    6: 'sexta',
    7: 'sabado'
}
day_name_map = F.create_map([F.lit(k_v) for kv in day_name_dict.items() for k_v in kv])

month_name_dict = {
    1: 'janeiro',
    2: 'fevereiro',
    3: 'marco', # proposital o 'c'
    4: 'abril',
    5: 'maio',
    6: 'julho',
    7: 'julho',
    8: 'agosto',
    9: 'setembro',
    10: 'outubro',
    11: 'novembro',
    12: 'dezembro'
}
month_name_map = F.create_map([F.lit(k_v) for kv in month_name_dict.items() for k_v in kv])

dm_tempo = (
    spark.range(1)  # cria apenas 1 linha
    .withColumn("sk_tempo",F.sequence(
            F.to_date(F.lit(data_inicio)),
            F.to_date(F.lit(data_fim)),
            F.expr("interval 1 day")
        )
    )
    .withColumn('sk_tempo', F.explode(F.expr(f"sequence(to_timestamp('{data_inicio}'), to_timestamp('{data_fim}'), interval 1 day)")))
    .withColumn('sk_tempo', F.col('sk_tempo').cast('date'))
    .withColumn('ano', F.year('sk_tempo'))
    .withColumn('mes', F.month('sk_tempo'))
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn('trimestre', F.ceil(F.col('mes') / 3))
    .withColumn('dia', F.dayofmonth('sk_tempo'))
    .withColumn('dia_semana_num', F.dayofweek('sk_tempo'))
    .withColumn('dia_semana_nome', day_name_map[F.col('dia_semana_num')])
    .withColumn('mes_nome', month_name_map[F.col('mes')])
    .withColumn('eh_fim_de_semana', F.when(F.col('dia_semana_num').isin(1, 7), 'sim').otherwise('nao'))
    .withColumn('data_ingestao', F.current_timestamp())
    )
(
    dm_tempo.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalogo}.{gold_schema}.dm_tempo")
)
print('dimension "dm_tempo" created')

# FATO VENDAS GERAL
s_ped_total = spark.read.table(f'{catalogo}.{silver_schema}.ft_pedido_total').alias('s_ped_total')
s_i_pd = spark.read.table(f'{catalogo}.{silver_schema}.ft_itens_pedidos').alias('s_i_pd')
s_pd = spark.read.table(f'{catalogo}.{silver_schema}.ft_pedidos').alias('s_pd')
s_ct = spark.read.table(f'{catalogo}.{silver_schema}.dm_cotacao_dolar').alias('s_ct')
s_aval_ped = spark.read.table(f'{catalogo}.{silver_schema}.ft_avaliacoes_pedidos').alias('s_aval_ped')

vendas_geral = (
    s_ped_total
    .join(s_i_pd, on='id_pedido')
    .join(s_pd, on='id_pedido')
    .join(s_ct, F.col('s_ct.data') == F.col('s_ped_total.data_pedido'))
    .join((
        s_aval_ped
        .groupBy('id_pedido')
        .agg(
            F.avg('avaliacao').alias('avaliacao_media')
            )
        )
        .alias('s_aval_ped'),
        on='id_pedido'
    )
    .select(
        's_ped_total.id_pedido',
        's_i_pd.id_item',
        F.col('s_ped_total.id_consumidor').alias('fk_consumidor'),
        F.col('s_i_pd.id_produto').alias('fk_produto'),
        F.col('s_i_pd.id_vendedor').alias('fk_vendedor'),
        F.col('s_ped_total.data_pedido').alias('fk_tempo'),
        F.col('s_ped_total.status').alias('status_pedido'),
        F.col('s_pd.tempo_entrega_dias').cast('int').alias('tempo_entrega_dias'),
        F.col('s_pd.entrega_no_prazo').alias('entrega_no_prazo'),
        F.col('s_i_pd.preco_BRL').alias('valor_produto_brl').cast(DecimalType(12, 2)),
        F.col('s_i_pd.preco_frete').alias('valor_frete_brl').cast(DecimalType(12, 2)),
        F.col('s_ct.cotacao_dolar').alias('cotacao_dolar').cast(DecimalType(8,4)),
        F.col('s_aval_ped.avaliacao_media').alias('avaliacao_media').cast(DecimalType(3, 2))
    )
    # diferente de s_ped_total.valor_total_pago_brl
    .withColumn('valor_total_item_brl',
                (F.col('valor_produto_brl') + F.col('valor_frete_brl')).cast(DecimalType(12, 2))
                )
    .withColumn('valor_produto_usd',
                (F.col('valor_produto_brl') / F.col('cotacao_dolar')).cast(DecimalType(12, 2))
                )
    .withColumn('valor_frete_usd',
                (F.col('valor_frete_brl') / F.col('cotacao_dolar')).cast(DecimalType(12, 2))
                )
    .withColumn('valor_total_item_usd',
                (F.col('valor_total_item_brl') / F.col('cotacao_dolar')).cast(DecimalType(12, 2))
                )
    .withColumn('data_ingestao', F.current_timestamp())
    )
(
    vendas_geral.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalogo}.{gold_schema}.ft_vendas_geral")
)
print('table "ft_vendas_geral" created')


# ft produtos silver to gold
s_prod = spark.read.table(f'{catalogo}.{silver_schema}.ft_produtos').alias('s_prod')
s_prod = s_prod.withColumn('data_ingestao', F.current_timestamp())
(
    s_prod.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true") 
    .saveAsTable(f"{catalogo}.{gold_schema}.ft_produtos")
)

In [0]:
%sql
CREATE OR REPLACE VIEW rocketlab.gold.view_vendas_por_periodo AS
SELECT 
    dmt.ano,
    dmt.trimestre,
    dmt.mes,
    dmt.mes_nome,
    dmt.dia,
    dmt.dia_semana_num,
    vg.total_pedidos,
    vg.total_itens,
    vg.receita_total_brl,
    vg.receita_total_usd,
    vg.ticket_medio_brl,
    vg.avaliacao_media,
    current_timestamp AS data_ingestao
FROM (
      --Agregacao do fato ft_vendas_geral
        SELECT
            fk_tempo,
            CAST(SUM(valor_total_item_brl) AS BIGINT) AS receita_total_brl,
            CAST(SUM(valor_total_item_usd) AS BIGINT) AS receita_total_usd,
            COUNT(DISTINCT id_pedido) AS total_pedidos,
            COUNT(id_item) AS total_itens,
            CAST(
                AVG(
                    CASE
                        WHEN status_pedido IN ('entregue', 'enviado') THEN avaliacao_media
                        ELSE 0
                    END
                ) AS DECIMAL(3,2)
            ) AS avaliacao_media,
            AVG(valor_produto_brl) AS ticket_medio_brl
        FROM rocketlab.gold.ft_vendas_geral
        GROUP BY fk_tempo
     ) AS vg
JOIN rocketlab.gold.dm_tempo AS dmt
ON vg.fk_tempo = dmt.sk_tempo

In [0]:
%sql
/*
 Não faz mt sentido querer peso medio, visto que cada item de ft_produtos é um produto unico (verificado: Total de linhas: 32951, Total de valores distintos: 32951)
*/
CREATE OR REPLACE VIEW rocketlab.gold.view_top_produto as
SELECT 
    p.id_produto as id_produto,
    p.categoria_produto as categoria_produto,
    CAST(COUNT(vg.id_pedido) as bigint) as quantidade_vendida,
    CAST(COUNT(DISTINCT vg.id_pedido) as bigint) as total_pedidos,
    SUM(vg.valor_total_item_brl) as receita_total_brl,
    SUM(vg.valor_total_item_usd) as receita_total_usd,
    AVG(vg.valor_produto_brl) as preco_medio_brl,
    CAST(AVG(vg.avaliacao_media) as DECIMAL(3,2)) as avaliacao_media,
    AVG(p.peso_produto_gramas) as peso_medio_gramas,
    current_timestamp() as data_ingestao
FROM rocketlab.gold.ft_vendas_geral as vg
JOIN rocketlab.gold.ft_produtos as p
ON vg.fk_produto = p.id_produto
GROUP BY p.id_produto, p.categoria_produto

In [0]:
%sql
CREATE OR REPLACE VIEW rocketlab.gold.view_vendas_produtos_esteticos AS
SELECT 
    t.ano as ano, 
    t.mes as mes,
    p.categoria_produto as categoria_produto,
    CAST(COUNT(DISTINCT vg.id_pedido) as BIGINT) as total_pedidos,
    CAST(COUNT(vg.id_pedido) as BIGINT) as total_itens_vendidos,
    SUM(vg.valor_total_item_brl) as receita_total_brl,
    SUM(vg.valor_total_item_usd) as receita_total_usd,
    AVG(vg.valor_produto_brl) as ticket_medio_brl,
    AVG(vg.valor_produto_usd) as ticket_medio_usd,
    CAST(AVG(vg.avaliacao_media) as DECIMAL(3,2)) as avaliacao_media,
    current_timestamp() as data_ingestao
FROM rocketlab.gold.ft_vendas_geral as vg
JOIN rocketlab.gold.ft_produtos as p
ON vg.fk_produto = p.id_produto
JOIN rocketlab.gold.dm_tempo as t
ON vg.fk_tempo = t.sk_tempo
WHERE lower(p.categoria_produto) LIKE '%fashion%'
GROUP BY p.categoria_produto, t.ano, t.mes, t.sk_tempo

In [0]:
%sql
SELECT *
FROM (
    SELECT 
        dia_semana_num,
        SUM(receita_total_brl) AS receita_total
    FROM rocketlab.gold.view_vendas_por_periodo
    GROUP BY dia_semana_num
) t
ORDER BY receita_total DESC
LIMIT 1;


In [0]:
%sql
SELECT mes, ROUND(ticket_medio_brl, 2)
FROM (
    SELECT 
        mes,
        AVG(ticket_medio_brl) AS ticket_medio_brl
    FROM rocketlab.gold.view_vendas_por_periodo
    WHERE ano = (SELECT MAX(ano) FROM rocketlab.gold.view_vendas_por_periodo)
    GROUP BY mes
) t
ORDER BY ticket_medio_brl DESC
LIMIT 1;

In [0]:
%sql
WITH cte_fashion AS (
   SELECT *
   FROM rocketlab.gold.view_vendas_produtos_esteticos
)
SELECT *
FROM cte_fashion;